In [2]:
from plugins.brew_project.libs.utils.utils import DeltaSpark

In [4]:
spark = DeltaSpark().initialize()


2024-02-03T17:35:18 | STARTING SPARK |===================================================================================
2024-02-03T17:35:18 | SPARK INITIALIZED |================================================================================


In [5]:
df = spark.read.format("delta").load("./brew_project/warehouse/silver.db/brewery")

In [6]:
df.createTempView('df')

In [7]:
spark.sql("""
SELECT id
FROM df
GROUP BY id
HAVING COUNT(id) > 1
""").show(truncate=False)

+---+
|id |
+---+
+---+


In [8]:
brewey_city = spark.read.format("delta").load("./brew_project/warehouse/gold.db/gold_d_brewery_city")
brewey_country = spark.read.format("delta").load("./brew_project/warehouse/gold.db/gold_d_brewery_country")
brewey_state = spark.read.format("delta").load("./brew_project/warehouse/gold.db/gold_d_brewery_state")
brewey = spark.read.format("delta").load("./brew_project/warehouse/gold.db/gold_f_brewery")
# ----------------------------------------------------------------------------------------------------------
brewey_city.createOrReplaceTempView('city')
brewey_country.createOrReplaceTempView('country')
brewey_state.createOrReplaceTempView('state')
brewey.createOrReplaceTempView('fact')

In [20]:
_sql = """
SELECT FACT.id             AS id,
       FACT.name,
       FACT.brewery_type,
       FACT.latitude,
       FACT.longitude,
       FACT.phone,
       FACT.address,
       DIM_COUNTRY.country AS country,
       DIM_STATE.state     AS state,
       DIM_CITY.city       AS city 

FROM fact AS FACT

LEFT JOIN country AS DIM_COUNTRY ON FACT.p_country = DIM_COUNTRY.PK
LEFT JOIN state   AS DIM_STATE   ON FACT.p_state = DIM_STATE.PK
LEFT JOIN city    AS DIM_CITY    ON FACT.p_city = DIM_CITY.PK
"""

df = spark.sql(_sql)

In [21]:
df.show()

+--------------------+--------------------+------------+---------+---------+----------+--------------------+-------------+-------------+------------+
|                  id|                name|brewery_type| latitude|longitude|     phone|             address|      country|        state|        city|
+--------------------+--------------------+------------+---------+---------+----------+--------------------+-------------+-------------+------------+
|0efdefd9-f325-479...|Bad Joker Brewing...|       micro| 41.59874| 41.59874|3075225824|830 Powerhouse Dr | |United States|      Wyoming|Rock Springs|
|281a2d47-03db-45a...|Poseidon Brewing ...|       micro|34.244152|34.244152|8054770239|5777 Olivas Park ...|United States|   California|     Ventura|
|1b089764-3735-43f...|Gillette Brewing ...|      closed|44.292038|44.292038|3076708948|301 S Gillette Av...|United States|      Wyoming|    Gillette|
|044daf89-a298-426...|Scrappy Punk Brewing|       micro|47.913654|47.913654|5038101655|9029A 112th D